In [79]:
import pandas as pd
import mysql.connector

import numpy as np

### Cargar nuestra base de datos con un csv a partir de la librería mysql.connector 

Antes de cargar los datos, veamos los valores únicos de cada columna para revisar el diseño de la base de datos

In [80]:
# Leer el CSV
df_sale = pd.read_csv("madrid_sale_properties_cleaned.csv")

In [10]:
df_sale.sample(3)

,price_eur,barrio,distrito,latitude,longitude,energy_consumption_rating,energy_consumption_value,energy_emissions_rating,energy_emissions_value,adaptado a personas con movilidad reducida,...,piscina,planta,portero automático,puerta blindada,sistema de seguridad,superficie construida,superficie útil,terraza,trastero,vidrios dobles
435,327000.0,Quintana,Ciudad Lineal,40.433745,-3.648607,E,246 kWh/m² año,E,42 Kg CO₂/m² año,0.0,...,False,8.0,False,False,False,51.0,51.0,False,False,False
737,875000.0,Trafalgar,Chamberí,40.432340,-3.703750,False,NaN,False,NaN,0.0,...,False,0.0,False,False,False,98.0,98.0,False,False,False
1605,1900000.0,Castellana,Salamanca,40.438092,-3.688786,False,NaN,False,NaN,0.0,...,False,8.0,False,False,False,170.0,155.0,False,False,False


In [81]:
# Leer el CSV
df_rental = pd.read_csv("madrid_rental_properties_cleaned.csv")

In [12]:
df_rental.sample(3)

,rent_eur_per_month,barrio,distrito,latitude,longitude,listing_type,energy_consumption_rating,energy_emissions_rating,adaptado a personas con movilidad reducida,aire acondicionado,...,planta,portero automático,puerta blindada,referencia,sistema de seguridad,superficie construida,superficie útil,terraza,trastero,vidrios dobles
1623,2800.0,Palomas,Hortaleza,40.453029,-3.618988,rental,False,False,0.0,False,...,4.0,False,False,I-03003Y-W-03003Y,False,164.0,164.0,False,False,False
2089,1500.0,Valverde,Fuencarral-El Pardo,40.486070,-3.692651,rental,False,False,0.0,True,...,6.0,False,True,22067048,False,110.0,90.0,False,False,False
471,3045.0,Universidad-Malasaña,Centro,40.426859,-3.708706,rental,C,E,0.0,False,...,2.0,False,False,CELEBES,False,140.0,140.0,False,False,False


Identificador de cada piso 

In [17]:
df_rental.shape

(2185, 34)

In [15]:
df_rental["referencia"].unique

<bound method Series.unique of 0                  6013
1                  3893
2                  4411
3                 20815
4       9635001141/9635
             ...       
2180             001913
2181            1101915
2182            1101712
2183            1101954
2184             209599
Name: referencia, Length: 2185, dtype: object>

antigüedad

In [20]:
df_rental["antigüedad"].unique()

array([nan, 'Más de 50 años', 'Entre 20 y 30 años', 'Entre 30 y 50 años',
       'Entre 5 y 10 años', 'Menos de 5 años', 'Entre 10 y 20 años'],
      dtype=object)

In [21]:
df_sale["antigüedad"].unique()

array([nan, 'Más de 50 años', 'Entre 20 y 30 años', 'Entre 30 y 50 años',
       'Menos de 5 años', 'Entre 10 y 20 años', 'Entre 5 y 10 años'],
      dtype=object)

conservacion

planta

exterior

In [23]:
df_rental.columns

Index(['rent_eur_per_month', 'barrio', 'distrito', 'latitude', 'longitude',
       'listing_type', 'energy_consumption_rating', 'energy_emissions_rating',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'cocina equipada', 'conservación', 'exterior', 'garaje',
       'habitaciones', 'jardín', 'piscina', 'planta', 'portero automático',
       'puerta blindada', 'referencia', 'sistema de seguridad',
       'superficie construida', 'superficie útil', 'terraza', 'trastero',
       'vidrios dobles'],
      dtype='object')

In [24]:
rental = ['energy_consumption_rating', 'energy_emissions_rating',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'cocina equipada', 'conservación', 'exterior', 'garaje',
       'habitaciones', 'jardín', 'piscina', 'planta', 'portero automático',
       'puerta blindada', 'referencia', 'sistema de seguridad',
       'superficie construida', 'superficie útil', 'terraza', 'trastero',
       'vidrios dobles']

In [25]:
[df_rental[columna].unique() for columna in rental]

[array(['False', 'D', 'E', 'C', 'F', 'G', 'B', 'A'], dtype=object),
 array(['False', 'D', 'E', 'C', 'G', 'F', 'B', 'A'], dtype=object),
 array([0., 1.]),
 array([ True, False]),
 array(['No', 'Si', 'Semi'], dtype=object),
 array([nan, 'Más de 50 años', 'Entre 20 y 30 años', 'Entre 30 y 50 años',
        'Entre 5 y 10 años', 'Menos de 5 años', 'Entre 10 y 20 años'],
       dtype=object),
 array([False,  True]),
 array(['False', '1.0'], dtype=object),
 array([5, 1, 2, 4, 3, 6, 7, 8], dtype=int64),
 array([ True, False]),
 array(['False', '1.0'], dtype=object),
 array(['True', 'False', 'Independiente', 'Equipada',
        'Amueblada y equipada', 'Equipada e independiente.',
        'Cocina amueblada', 'Cocina americana', 'Completamente equipada',
        'Completamente equipada integrada al salón',
        'Independiente - amueblada con elec.',
        'Independiente totalmente equipada y amueblada', 'Cocina equipada',
        'Americana', 'Americana - amueblada con elec.', 'Amueblada',
 

In [18]:
df_sale.columns

Index(['price_eur', 'barrio', 'distrito', 'latitude', 'longitude',
       'energy_consumption_rating', 'energy_consumption_value',
       'energy_emissions_rating', 'energy_emissions_value',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'conservación', 'exterior', 'garaje', 'habitaciones',
       'jardín', 'piscina', 'planta', 'portero automático', 'puerta blindada',
       'sistema de seguridad', 'superficie construida', 'superficie útil',
       'terraza', 'trastero', 'vidrios dobles'],
      dtype='object')

In [33]:
sale = ['energy_consumption_rating',
       'energy_emissions_rating',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'conservación', 'exterior', 'garaje', 'habitaciones',
       'jardín', 'piscina', 'planta', 'portero automático', 'puerta blindada',
       'sistema de seguridad',
       'terraza', 'trastero', 'vidrios dobles']

In [34]:
[df_sale[columna].unique() for columna in sale]

[array(['False', 'E', 'C', 'D', 'A', 'F', 'G', 'B'], dtype=object),
 array(['False', 'D', 'E', 'A', 'F', 'C', 'B', 'G'], dtype=object),
 array([0., 1.]),
 array([ True, False]),
 array([ True, False]),
 array([nan, 'Más de 50 años', 'Entre 20 y 30 años', 'Entre 30 y 50 años',
        'Menos de 5 años', 'Entre 10 y 20 años', 'Entre 5 y 10 años'],
       dtype=object),
 array([False,  True]),
 array(['1.0', 'False'], dtype=object),
 array([ 5,  2,  1,  4,  3,  6,  7, 17,  8, 11,  9, 13, 10, 14],
       dtype=int64),
 array([ True, False]),
 array(['False', 'True', 'Preinstalación'], dtype=object),
 array(['A estrenar', 'En buen estado', 'Reformado', 'A reformar'],
       dtype=object),
 array([False,  True]),
 array([False,  True]),
 array([ 4.,  5.,  2.,  3., 14.,  6.,  1., 12.,  8., nan, 24.,  7., 11.,
         9., 10., 17., 13., 20., 36.]),
 array([False,  True]),
 array([False,  True]),
 array([ 1.,  2.,  3.,  0.,  5.,  6.,  4.,  7.,  8., 10., -1.,  9., 12.,
        11., 20., 14., 16

In [40]:
df_sale['energy_consumption_value'].unique()

array([nan, '151 kWh/m² año', '152 kWh/m² año', '30 kWh/m² año',
       '100 kWh/m² año', '300 kWh/m² año', '219 kWh/m² año',
       '24 kWh/m² año', '124 kWh/m² año', '346 kWh/m² año',
       '95 kWh/m² año', '246 kWh/m² año', '228 kWh/m² año',
       '195 kWh/m² año', '307 kWh/m² año', '130 kWh/m² año',
       '63 kWh/m² año', '309 kWh/m² año', '199 kWh/m² año',
       '148 kWh/m² año', '134 kWh/m² año', '150 kWh/m² año',
       '121 kWh/m² año', '98 kWh/m² año', '169 kWh/m² año',
       '104 kWh/m² año', '179 kWh/m² año', '167 kWh/m² año',
       '298 kWh/m² año', '244 kWh/m² año', '165 kWh/m² año',
       '999 kWh/m² año', '254 kWh/m² año', '242 kWh/m² año',
       '146 kWh/m² año', '226 kWh/m² año', '50 kWh/m² año',
       '361 kWh/m² año', '86 kWh/m² año', '192 kWh/m² año',
       '188 kWh/m² año', '48 kWh/m² año', '196 kWh/m² año',
       '204 kWh/m² año', '75 kWh/m² año', '174 kWh/m² año',
       '198 kWh/m² año', '315 kWh/m² año', '194 kWh/m² año',
       '136 kWh/m² año', '26

In [36]:
df_rental["energy_consumption_rating"].unique()

array(['False', 'D', 'E', 'C', 'F', 'G', 'B', 'A'], dtype=object)

In [41]:
df_rental["latitude"]

0       40.434281
1       40.438612
2       40.435237
3       40.429047
4       40.422607
          ...    
2180    40.516174
2181    40.516126
2182    40.521171
2183    40.519891
2184    40.507050
Name: latitude, Length: 2185, dtype: float64

In [42]:
df_sale.columns

Index(['price_eur', 'barrio', 'distrito', 'latitude', 'longitude',
       'energy_consumption_rating', 'energy_consumption_value',
       'energy_emissions_rating', 'energy_emissions_value',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'conservación', 'exterior', 'garaje', 'habitaciones',
       'jardín', 'piscina', 'planta', 'portero automático', 'puerta blindada',
       'sistema de seguridad', 'superficie construida', 'superficie útil',
       'terraza', 'trastero', 'vidrios dobles'],
      dtype='object')

In [43]:
df_rental.columns

Index(['rent_eur_per_month', 'barrio', 'distrito', 'latitude', 'longitude',
       'listing_type', 'energy_consumption_rating', 'energy_emissions_rating',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'cocina equipada', 'conservación', 'exterior', 'garaje',
       'habitaciones', 'jardín', 'piscina', 'planta', 'portero automático',
       'puerta blindada', 'referencia', 'sistema de seguridad',
       'superficie construida', 'superficie útil', 'terraza', 'trastero',
       'vidrios dobles'],
      dtype='object')

conectamos con la base de datos para hacer la insercion

In [331]:
conn.close()
cursor.close()

True

In [332]:
# Conexión a MySQL
conn = mysql.connector.connect(
    host="localhost", 
    user="root", 
    password="contraseña",
    database="pisos2"
)

In [333]:
cursor = conn.cursor()

Una vez conectada la base de datos vamos a concatenar los dos dataframe, para ello necesitamos unificar los campos 

### Unificar precio y tipo

In [334]:
df_rental = df_rental.rename(
    columns={"rent_eur_per_month": "price_current"}
)

In [335]:
df_rental.columns

Index(['price_current', 'price_kind', 'listing_type', 'barrio', 'distrito',
       'latitude', 'longitude', 'energy_consumption_rating',
       'energy_emissions_rating', 'adaptado a personas con movilidad reducida',
       'aire acondicionado', 'amueblado', 'antigüedad', 'ascensor', 'balcón',
       'baños', 'calefacción', 'chimenea', 'cocina equipada', 'conservación',
       'exterior', 'garaje', 'habitaciones', 'jardín', 'piscina', 'planta',
       'portero automático', 'puerta blindada', 'sistema de seguridad',
       'superficie construida', 'superficie útil', 'terraza', 'trastero',
       'vidrios dobles', 'referencia'],
      dtype='object')

In [336]:
df_rental["price_kind"]   = "rent_month"
df_rental["listing_type"] = "rental"    # para identificarlos mejor 

In [337]:
df_rental["price_kind"].unique()

array(['rent_month'], dtype=object)

In [338]:
df_rental["listing_type"].unique()

array(['rental'], dtype=object)

In [339]:
df_sale = df_sale.rename(
    columns={"price_eur": "price_current"}
)

In [340]:
df_sale.columns

Index(['price_current', 'price_kind', 'listing_type', 'barrio', 'distrito',
       'latitude', 'longitude', 'energy_consumption_rating',
       'energy_emissions_rating', 'adaptado a personas con movilidad reducida',
       'aire acondicionado', 'amueblado', 'antigüedad', 'ascensor', 'balcón',
       'baños', 'calefacción', 'chimenea', 'cocina equipada', 'conservación',
       'exterior', 'garaje', 'habitaciones', 'jardín', 'piscina', 'planta',
       'portero automático', 'puerta blindada', 'sistema de seguridad',
       'superficie construida', 'superficie útil', 'terraza', 'trastero',
       'vidrios dobles', 'referencia'],
      dtype='object')

In [341]:
df_sale["price_kind"]   = "sale_price"
df_sale["listing_type"] = "sale"

In [342]:
df_sale["price_kind"].unique()

array(['sale_price'], dtype=object)

In [343]:
df_sale["listing_type"].unique()

array(['sale'], dtype=object)

### Las columnas de valor energetico de sale no las vamos a usar así que serán borradas

In [344]:
df_sale = df_sale.drop(columns=["energy_consumption_value", "energy_emissions_value"])

KeyError: "['energy_consumption_value', 'energy_emissions_value'] not found in axis"

In [ ]:
df_sale.columns

Index(['price_current', 'barrio', 'distrito', 'latitude', 'longitude',
       'energy_consumption_rating', 'energy_emissions_rating',
       'adaptado a personas con movilidad reducida', 'aire acondicionado',
       'amueblado', 'antigüedad', 'ascensor', 'balcón', 'baños', 'calefacción',
       'chimenea', 'conservación', 'exterior', 'garaje', 'habitaciones',
       'jardín', 'piscina', 'planta', 'portero automático', 'puerta blindada',
       'sistema de seguridad', 'superficie construida', 'superficie útil',
       'terraza', 'trastero', 'vidrios dobles', 'price_kind', 'listing_type'],
      dtype='object')

### Necesitamos crear referencias unicas para ventas

In [ ]:
df_rental["referencia"]

0                  6013
1                  3893
2                  4411
3                 20815
4       9635001141/9635
             ...       
2180             001913
2181            1101915
2182            1101712
2183            1101954
2184             209599
Name: referencia, Length: 2185, dtype: object

In [ ]:
["ref"+str(i) for i in range(len(df_sale))]

['ref0',
 'ref1',
 'ref2',
 'ref3',
 'ref4',
 'ref5',
 'ref6',
 'ref7',
 'ref8',
 'ref9',
 'ref10',
 'ref11',
 'ref12',
 'ref13',
 'ref14',
 'ref15',
 'ref16',
 'ref17',
 'ref18',
 'ref19',
 'ref20',
 'ref21',
 'ref22',
 'ref23',
 'ref24',
 'ref25',
 'ref26',
 'ref27',
 'ref28',
 'ref29',
 'ref30',
 'ref31',
 'ref32',
 'ref33',
 'ref34',
 'ref35',
 'ref36',
 'ref37',
 'ref38',
 'ref39',
 'ref40',
 'ref41',
 'ref42',
 'ref43',
 'ref44',
 'ref45',
 'ref46',
 'ref47',
 'ref48',
 'ref49',
 'ref50',
 'ref51',
 'ref52',
 'ref53',
 'ref54',
 'ref55',
 'ref56',
 'ref57',
 'ref58',
 'ref59',
 'ref60',
 'ref61',
 'ref62',
 'ref63',
 'ref64',
 'ref65',
 'ref66',
 'ref67',
 'ref68',
 'ref69',
 'ref70',
 'ref71',
 'ref72',
 'ref73',
 'ref74',
 'ref75',
 'ref76',
 'ref77',
 'ref78',
 'ref79',
 'ref80',
 'ref81',
 'ref82',
 'ref83',
 'ref84',
 'ref85',
 'ref86',
 'ref87',
 'ref88',
 'ref89',
 'ref90',
 'ref91',
 'ref92',
 'ref93',
 'ref94',
 'ref95',
 'ref96',
 'ref97',
 'ref98',
 'ref99',
 'ref100',

In [ ]:
df_sale["referencia"] = ["ref"+str(i) for i in range(len(df_sale))]

In [ ]:
df_sale["referencia"]

0          ref0
1          ref1
2          ref2
3          ref3
4          ref4
         ...   
2997    ref2997
2998    ref2998
2999    ref2999
3000    ref3000
3001    ref3001
Name: referencia, Length: 3002, dtype: object

### Definir columnas finales en orden y concatenar

In [ ]:
common_cols = [
  "price_current", "price_kind", "listing_type",
  "barrio", "distrito", "latitude", "longitude",
  "energy_consumption_rating", "energy_emissions_rating",
  "adaptado a personas con movilidad reducida",
  "aire acondicionado",
  "amueblado",
  "antigüedad",
  "ascensor", "balcón", "baños", "calefacción", "chimenea",
  "cocina equipada", "conservación", "exterior", "garaje",
  "habitaciones", "jardín", "piscina", "planta",
  "portero automático", "puerta blindada",
  "sistema de seguridad",
  "superficie construida", "superficie útil",
  "terraza", "trastero", "vidrios dobles",
  "referencia"
]

In [ ]:
for df in (df_rental, df_sale):
    for col in common_cols:
        if col not in df.columns:
            df[col] = np.nan

In [ ]:
df_rental = df_rental[common_cols]
df_sale   = df_sale[common_cols]

In [ ]:
df = pd.concat([df_rental, df_sale], ignore_index=True)
df = df.drop_duplicates()

In [ ]:
df.shape

(5186, 35)

In [ ]:
df_sale.shape + df_rental.shape

(3002, 35, 2185, 35)

In [ ]:
df.tail(2)

,price_current,price_kind,listing_type,barrio,distrito,latitude,longitude,energy_consumption_rating,energy_emissions_rating,adaptado a personas con movilidad reducida,...,planta,portero automático,puerta blindada,sistema de seguridad,superficie construida,superficie útil,terraza,trastero,vidrios dobles,referencia
5185,2490000.0,sale_price,sale,Trafalgar,Chamberí,40.435597,-3.700479,False,False,0.0,...,4.0,False,True,False,185.0,160.0,False,False,False,ref3000
5186,2460000.0,sale_price,sale,Sol,Centro,40.415001,-3.706658,False,False,1.0,...,5.0,False,False,False,139.0,139.0,False,False,False,ref3001


### Insertar datos en la base de datos

 La estrategia de trabajo será ir creando los df correspondientes a cada tabla de sql, rellenando las tablas independientes primero. El orden es el siguiente

age_range
– Inserta todos los rangos de antigüedad.
– (Si lo tienes en CSV, léelo primero y haz INSERT sobre age_range.)

feature_catalog
– Inserta tu lista de features (“ascensor”, “balcón”, …).
– También se puede cargar desde CSV si lo tienes en un fichero aparte.

location
– Inserta todos los barrios/distritos.

property
– Ahora que location y age_range existen, puedes insertar las viviendas.
– Usa el location_id y el age_range_id resultantes.

listing
– Con property_id ya presente, carga los anuncios.

rental_term
– Solo haces INSERT si listing_type = 'rental'. Usa el listing_id correspondiente.

energy_certificate
– Solo para las propiedades que tengan certificado, usando su property_id.

property_feature
– Finalmente, para cada tupla (property_id, feature_id, valor) extraída de tu CSV de extras, haces INSERT.

### Tabla ``age_range``

In [ ]:
df["antigüedad"].dropna().unique().tolist()

['Más de 50 años',
 'Entre 20 y 30 años',
 'Entre 30 y 50 años',
 'Entre 5 y 10 años',
 'Menos de 5 años',
 'Entre 10 y 20 años']

In [ ]:
age_range = df["antigüedad"].dropna().unique().tolist()

In [ ]:
age_range

['Más de 50 años',
 'Entre 20 y 30 años',
 'Entre 30 y 50 años',
 'Entre 5 y 10 años',
 'Menos de 5 años',
 'Entre 10 y 20 años']

creamos la sentencia de sql

In [ ]:
sql_age = """INSERT INTO age_range(label) VALUES (%s)"""

In [ ]:
cursor.executemany(sql_age, [(label,) for label in age_range])

In [ ]:
conn.commit()

Guardamos el mapeo para saber que id tiene cada rango

In [ ]:
cursor.execute("SELECT * FROM age_range")

In [ ]:
ages = cursor.fetchall()
ages

[(6, 'Entre 10 y 20 años'),
 (2, 'Entre 20 y 30 años'),
 (3, 'Entre 30 y 50 años'),
 (4, 'Entre 5 y 10 años'),
 (1, 'Más de 50 años'),
 (5, 'Menos de 5 años')]

In [ ]:
age_map = {label: age_range_id for age_range_id, label in ages}
age_map

{'Entre 10 y 20 años': 6,
 'Entre 20 y 30 años': 2,
 'Entre 30 y 50 años': 3,
 'Entre 5 y 10 años': 4,
 'Más de 50 años': 1,
 'Menos de 5 años': 5}

### Tabla ``feature_map``

INSERT INTO feature_catalog(nombre) VALUES
  ('ascensor'),
  ('balcon'),
  ('calefaccion'),
  ('chimenea'),
  ('exterior'),
  ('garaje'),
  ('piscina'),
  ('trastero'),
  ('jardin'),
  ('adaptado_pmreducida'),
  ('aire_acondicionado'),
  ('puerta_blindada'),
  ('vidrios_dobles'),
  ('sistema_seguridad'),
  ('terraza');

In [ ]:
features = [
    'ascensor',
    'balcón',
    'calefacción',
    'chimenea',
    'exterior',
    'garaje',
    'piscina',
    'trastero',
    'jardín',
    'adaptado_pmreducida',
    'aire_acondicionado',
    'puerta_blindada',
    'vidrios_dobles',
    'sistema_seguridad',
    'terraza'
]

In [ ]:
sql_feature = """INSERT INTO feature_catalog(nombre) VALUES (%s)"""

In [ ]:
cursor.executemany(sql_feature, [(f,) for f in features])

In [ ]:
conn.commit()

Extraer el mapeo de la base de datos

In [ ]:
cursor.execute("SELECT * FROM feature_catalog")

In [ ]:
f = cursor.fetchall()
f

[(10, 'adaptado_pmreducida'),
 (11, 'aire_acondicionado'),
 (1, 'ascensor'),
 (2, 'balcón'),
 (3, 'calefacción'),
 (4, 'chimenea'),
 (5, 'exterior'),
 (6, 'garaje'),
 (9, 'jardín'),
 (7, 'piscina'),
 (12, 'puerta_blindada'),
 (14, 'sistema_seguridad'),
 (15, 'terraza'),
 (8, 'trastero'),
 (13, 'vidrios_dobles')]

In [ ]:
feature_map = {nombre: feature_id for feature_id, nombre in f}
feature_map

{'adaptado_pmreducida': 10,
 'aire_acondicionado': 11,
 'ascensor': 1,
 'balcón': 2,
 'calefacción': 3,
 'chimenea': 4,
 'exterior': 5,
 'garaje': 6,
 'jardín': 9,
 'piscina': 7,
 'puerta_blindada': 12,
 'sistema_seguridad': 14,
 'terraza': 15,
 'trastero': 8,
 'vidrios_dobles': 13}

### Tabla ``Location``

In [ ]:
df[["barrio","distrito"]].drop_duplicates()

,barrio,distrito
0,Castellana,Salamanca
1,Río Rosas,Chamberí
3,Fuente del Berro,Salamanca
4,Justicia-Chueca,Centro
5,Aravaca,Moncloa-Aravaca
...,...,...
2477,Portazgo,Puente de Vallecas
2493,Apóstol Santiago,Hortaleza
2926,Centro Histórico,Vicálvaro
3557,Horcajo,Moratalaz


In [ ]:
df_location = df[["barrio","distrito"]].drop_duplicates()
df_location

,barrio,distrito
0,Castellana,Salamanca
1,Río Rosas,Chamberí
3,Fuente del Berro,Salamanca
4,Justicia-Chueca,Centro
5,Aravaca,Moncloa-Aravaca
...,...,...
2477,Portazgo,Puente de Vallecas
2493,Apóstol Santiago,Hortaleza
2926,Centro Histórico,Vicálvaro
3557,Horcajo,Moratalaz


In [ ]:
df_location.isna().sum()

barrio      0
distrito    0
dtype: int64

In [ ]:
df_location = df_location.dropna()

Pasamos a tupla para poder insertar

In [ ]:
locations = [tuple(x) for x in df_location.values]
locations

[('Castellana', 'Salamanca'),
 ('Río Rosas', 'Chamberí'),
 ('Fuente del Berro', 'Salamanca'),
 ('Justicia-Chueca', 'Centro'),
 ('Aravaca', 'Moncloa-Aravaca'),
 ('Rejas', 'San Blas'),
 ('Berruguete', 'Tetuán'),
 ('Simancas', 'San Blas'),
 ('Ibiza', 'Retiro'),
 ('Mirasierra-Arroyo del Fresno', 'Fuencarral-El Pardo'),
 ('Lista', 'Salamanca'),
 ('Recoletos', 'Salamanca'),
 ('Buenavista', 'Carabanchel'),
 ('Puerta del Ángel', 'Latina'),
 ('Pinar del Rey', 'Hortaleza'),
 ('El Viso', 'Chamartín'),
 ('Atocha', 'Arganzuela'),
 ('Almagro', 'Chamberí'),
 ('Peñagrande', 'Fuencarral-El Pardo'),
 ('Niño Jesús', 'Retiro'),
 ('Imperial', 'Arganzuela'),
 ('Trafalgar', 'Chamberí'),
 ('Palacio', 'Centro'),
 ('Valdeacederas', 'Tetuán'),
 ('Estrella', 'Retiro'),
 ('El Cañaveral', 'Vicálvaro'),
 ('Chopera', 'Arganzuela'),
 ('Jerónimos', 'Retiro'),
 ('Arapiles', 'Chamberí'),
 ('Cuatro Caminos', 'Tetuán'),
 ('Almenara', 'Tetuán'),
 ('Opañel', 'Carabanchel'),
 ('San Pascual', 'Ciudad Lineal'),
 ('Universidad-M

In [ ]:
sql_loc = """INSERT INTO location (barrio, distrito) VALUES (%s, %s)"""

In [ ]:
cursor.executemany(sql_loc, locations)

In [ ]:
conn.commit()

mapeo de location_id

In [ ]:
cursor.execute("SELECT * FROM location")

In [ ]:
l = cursor.fetchall()
l

[(1, 'Castellana', 'Salamanca'),
 (2, 'Río Rosas', 'Chamberí'),
 (3, 'Fuente del Berro', 'Salamanca'),
 (4, 'Justicia-Chueca', 'Centro'),
 (5, 'Aravaca', 'Moncloa-Aravaca'),
 (6, 'Rejas', 'San Blas'),
 (7, 'Berruguete', 'Tetuán'),
 (8, 'Simancas', 'San Blas'),
 (9, 'Ibiza', 'Retiro'),
 (10, 'Mirasierra-Arroyo del Fresno', 'Fuencarral-El Pardo'),
 (11, 'Lista', 'Salamanca'),
 (12, 'Recoletos', 'Salamanca'),
 (13, 'Buenavista', 'Carabanchel'),
 (14, 'Puerta del Ángel', 'Latina'),
 (15, 'Pinar del Rey', 'Hortaleza'),
 (16, 'El Viso', 'Chamartín'),
 (17, 'Atocha', 'Arganzuela'),
 (18, 'Almagro', 'Chamberí'),
 (19, 'Peñagrande', 'Fuencarral-El Pardo'),
 (20, 'Niño Jesús', 'Retiro'),
 (21, 'Imperial', 'Arganzuela'),
 (22, 'Trafalgar', 'Chamberí'),
 (23, 'Palacio', 'Centro'),
 (24, 'Valdeacederas', 'Tetuán'),
 (25, 'Estrella', 'Retiro'),
 (26, 'El Cañaveral', 'Vicálvaro'),
 (27, 'Chopera', 'Arganzuela'),
 (28, 'Jerónimos', 'Retiro'),
 (29, 'Arapiles', 'Chamberí'),
 (30, 'Cuatro Caminos', 'Tet

In [ ]:
loc_map = {
    (barrio, distrito): location_id
    for location_id, barrio, distrito in l
}

loc_map

{('Castellana', 'Salamanca'): 1,
 ('Río Rosas', 'Chamberí'): 2,
 ('Fuente del Berro', 'Salamanca'): 3,
 ('Justicia-Chueca', 'Centro'): 4,
 ('Aravaca', 'Moncloa-Aravaca'): 5,
 ('Rejas', 'San Blas'): 6,
 ('Berruguete', 'Tetuán'): 7,
 ('Simancas', 'San Blas'): 8,
 ('Ibiza', 'Retiro'): 9,
 ('Mirasierra-Arroyo del Fresno', 'Fuencarral-El Pardo'): 10,
 ('Lista', 'Salamanca'): 11,
 ('Recoletos', 'Salamanca'): 12,
 ('Buenavista', 'Carabanchel'): 13,
 ('Puerta del Ángel', 'Latina'): 14,
 ('Pinar del Rey', 'Hortaleza'): 15,
 ('El Viso', 'Chamartín'): 16,
 ('Atocha', 'Arganzuela'): 17,
 ('Almagro', 'Chamberí'): 18,
 ('Peñagrande', 'Fuencarral-El Pardo'): 19,
 ('Niño Jesús', 'Retiro'): 20,
 ('Imperial', 'Arganzuela'): 21,
 ('Trafalgar', 'Chamberí'): 22,
 ('Palacio', 'Centro'): 23,
 ('Valdeacederas', 'Tetuán'): 24,
 ('Estrella', 'Retiro'): 25,
 ('El Cañaveral', 'Vicálvaro'): 26,
 ('Chopera', 'Arganzuela'): 27,
 ('Jerónimos', 'Retiro'): 28,
 ('Arapiles', 'Chamberí'): 29,
 ('Cuatro Caminos', 'Tetuán'

### Tabla ``property`` 

In [ ]:
# eliminamos las filas donde no hay ni barrio ni distrito

df = df.dropna(subset=["barrio","distrito"], how="all")

In [ ]:
df_rental["amueblado"].unique()

array(['No', 'Si', 'Semi'], dtype=object)

In [ ]:
# la tabla amueblado no está unificada
df["amueblado"] = df["amueblado"].replace({True: "Si", False: "No"})

In [ ]:
df[[
    "barrio","distrito",
    "latitude","longitude",
    "referencia",
    "superficie construida","superficie útil",
    "habitaciones","baños","planta",
    "conservación","antigüedad",
    "amueblado","cocina equipada"
]].drop_duplicates()

,barrio,distrito,latitude,longitude,referencia,superficie construida,superficie útil,habitaciones,baños,planta,conservación,antigüedad,amueblado,cocina equipada
0,Castellana,Salamanca,40.434281,-3.686286,6013,272.0,272.0,4.0,5,6.0,A estrenar,NaN,No,True
1,Río Rosas,Chamberí,40.438612,-3.699901,3893,74.0,74.0,1.0,1,6.0,En buen estado,NaN,Si,True
2,Castellana,Salamanca,40.435237,-3.683515,4411,101.0,100.0,3.0,1,2.0,En buen estado,NaN,Si,False
3,Fuente del Berro,Salamanca,40.429047,-3.666474,20815,65.0,50.0,3.0,1,6.0,En buen estado,Más de 50 años,Si,Independiente
4,Justicia-Chueca,Centro,40.422607,-3.699196,9635001141/9635,83.0,77.0,2.0,2,5.0,En buen estado,Más de 50 años,Si,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5182,Portazgo,Puente de Vallecas,40.383712,-3.650452,ref2997,113.0,113.0,3.0,2,2.0,Reformado,Entre 30 y 50 años,Si,NaN
5183,Pueblo Nuevo,Ciudad Lineal,40.426333,-3.638205,ref2998,130.0,120.0,3.0,2,0.0,En buen estado,Más de 50 años,Si,NaN
5184,Valdemarín,Moncloa-Aravaca,40.465343,-3.762967,ref2999,239.0,239.0,4.0,4,2.0,En buen estado,Entre 20 y 30 años,Si,NaN
5185,Trafalgar,Chamberí,40.435597,-3.700479,ref3000,185.0,160.0,4.0,3,4.0,En buen estado,NaN,Si,NaN


In [ ]:
df_property = df[[
    "barrio","distrito",
    "latitude","longitude",
    "ref_unique",
    "superficie construida","superficie útil",
    "habitaciones","baños","planta",
    "conservación","antigüedad",
    "amueblado","cocina equipada"
]].drop_duplicates()

df_property

,barrio,distrito,latitude,longitude,ref_unique,superficie construida,superficie útil,habitaciones,baños,planta,conservación,antigüedad,amueblado,cocina equipada
0,Castellana,Salamanca,40.434281,-3.686286,6013,272.0,272.0,4.0,5,6.0,A estrenar,NaN,No,True
1,Río Rosas,Chamberí,40.438612,-3.699901,3893,74.0,74.0,1.0,1,6.0,En buen estado,NaN,Si,True
2,Castellana,Salamanca,40.435237,-3.683515,4411,101.0,100.0,3.0,1,2.0,En buen estado,NaN,Si,False
3,Fuente del Berro,Salamanca,40.429047,-3.666474,20815,65.0,50.0,3.0,1,6.0,En buen estado,Más de 50 años,Si,Independiente
4,Justicia-Chueca,Centro,40.422607,-3.699196,9635001141/9635,83.0,77.0,2.0,2,5.0,En buen estado,Más de 50 años,Si,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5182,Portazgo,Puente de Vallecas,40.383712,-3.650452,ref2997,113.0,113.0,3.0,2,2.0,Reformado,Entre 30 y 50 años,Si,NaN
5183,Pueblo Nuevo,Ciudad Lineal,40.426333,-3.638205,ref2998,130.0,120.0,3.0,2,0.0,En buen estado,Más de 50 años,Si,NaN
5184,Valdemarín,Moncloa-Aravaca,40.465343,-3.762967,ref2999,239.0,239.0,4.0,4,2.0,En buen estado,Entre 20 y 30 años,Si,NaN
5185,Trafalgar,Chamberí,40.435597,-3.700479,ref3000,185.0,160.0,4.0,3,4.0,En buen estado,NaN,Si,NaN


In [ ]:
df_property = df_property.rename(columns={
    "ref_unique":      "property_native_id",
    "superficie construida":  "superficie_construida",
    "superficie útil":        "superficie_util",
    "baños":                  "banos",
    "conservación":           "estado_conservacion",
    "cocina equipada":        "cocina_equipada"
})

In [ ]:
df_property.isna().sum()

barrio                      0
distrito                    0
latitude                    0
longitude                   0
property_native_id          0
superficie_construida      17
superficie_util            15
habitaciones                9
banos                       0
planta                      0
estado_conservacion         0
antigüedad               2489
amueblado                   0
cocina_equipada          3001
dtype: int64

In [ ]:
df_property = df_property.replace({np.nan: None})

In [ ]:
df_property

,barrio,distrito,latitude,longitude,property_native_id,superficie_construida,superficie_util,habitaciones,banos,planta,estado_conservacion,antigüedad,amueblado,cocina_equipada
0,Castellana,Salamanca,40.434281,-3.686286,6013,272.0,272.0,4.0,5,6.0,A estrenar,None,No,True
1,Río Rosas,Chamberí,40.438612,-3.699901,3893,74.0,74.0,1.0,1,6.0,En buen estado,None,Si,True
2,Castellana,Salamanca,40.435237,-3.683515,4411,101.0,100.0,3.0,1,2.0,En buen estado,None,Si,False
3,Fuente del Berro,Salamanca,40.429047,-3.666474,20815,65.0,50.0,3.0,1,6.0,En buen estado,Más de 50 años,Si,Independiente
4,Justicia-Chueca,Centro,40.422607,-3.699196,9635001141/9635,83.0,77.0,2.0,2,5.0,En buen estado,Más de 50 años,Si,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5182,Portazgo,Puente de Vallecas,40.383712,-3.650452,ref2997,113.0,113.0,3.0,2,2.0,Reformado,Entre 30 y 50 años,Si,None
5183,Pueblo Nuevo,Ciudad Lineal,40.426333,-3.638205,ref2998,130.0,120.0,3.0,2,0.0,En buen estado,Más de 50 años,Si,None
5184,Valdemarín,Moncloa-Aravaca,40.465343,-3.762967,ref2999,239.0,239.0,4.0,4,2.0,En buen estado,Entre 20 y 30 años,Si,None
5185,Trafalgar,Chamberí,40.435597,-3.700479,ref3000,185.0,160.0,4.0,3,4.0,En buen estado,None,Si,None


Contruimos la lista de tuplas para insertar

In [ ]:
property_records = []
for _, row in df_property.iterrows():
    loc_id = loc_map.get((row.barrio, row.distrito))
    age_id = age_map.get(row.antigüedad)

    habitaciones = int(row.habitaciones) if row.habitaciones is not None else None
    banos        = int(row.banos)         if row.banos        is not None else None
    planta       = int(row.planta)        if row.planta       is not None else None

    property_records.append((
        loc_id,
        row.latitude,
        row.longitude,
        row.property_native_id,
        row.superficie_construida,
        row.superficie_util,
        habitaciones,
        banos,
        planta,
        row.estado_conservacion,
        age_id,
        row.amueblado,
        row.cocina_equipada
    ))

In [ ]:
property_records

[(1,
  40.4342811,
  -3.6862864,
  '6013',
  272.0,
  272.0,
  4,
  5,
  6,
  'A estrenar',
  None,
  'No',
  'True'),
 (2,
  40.4386119,
  -3.6999008,
  '3893',
  74.0,
  74.0,
  1,
  1,
  6,
  'En buen estado',
  None,
  'Si',
  'True'),
 (1,
  40.4352372,
  -3.6835148,
  '4411',
  101.0,
  100.0,
  3,
  1,
  2,
  'En buen estado',
  None,
  'Si',
  'False'),
 (3,
  40.4290471,
  -3.6664739,
  '20815',
  65.0,
  50.0,
  3,
  1,
  6,
  'En buen estado',
  1,
  'Si',
  'Independiente'),
 (4,
  40.4226065,
  -3.6991962,
  '9635001141/9635',
  83.0,
  77.0,
  2,
  2,
  5,
  'En buen estado',
  1,
  'Si',
  'False'),
 (5,
  40.4446662,
  -3.7879384,
  'Pza_2dvac',
  90.0,
  90.0,
  2,
  1,
  2,
  'En buen estado',
  None,
  'No',
  'False'),
 (6,
  40.4430413,
  -3.5754374,
  '2209/48',
  81.0,
  62.0,
  1,
  1,
  0,
  'En buen estado',
  None,
  'No',
  'False'),
 (4,
  40.425045,
  -3.6990557,
  'EAT: 3062',
  28.0,
  28.0,
  1,
  1,
  0,
  'A estrenar',
  1,
  'Si',
  'Equipada'),
 (7,

In [ ]:
sql_property = """
        INSERT INTO property
        (location_id, latitude, longitude,
        property_native_id,
        superficie_construida, superficie_util,
        habitaciones, banos, planta,
        estado_conservacion, age_range_id,
        amueblado, cocina_equipada)
        VALUES
        (%s,%s,%s,
        %s,
        %s,%s,
        %s,%s,%s,
        %s,%s,
        %s,%s)
    """

In [ ]:
max(df["referencia"].agg(len))

C:\Users\marta\AppData\Local\Temp\ipykernel_25340\2231584434.py:1: FutureWarning: using <built-in function len> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  max(df["referencia"].agg(len))


50

In [ ]:
df["amueblado"].unique()

array(['No', 'Si', 'Semi'], dtype=object)

In [ ]:
df_rental["amueblado"].unique()

array(['No', 'Si', 'Semi'], dtype=object)

In [ ]:
df_sale["amueblado"].unique()

array([ True, False])

In [ ]:
cursor.executemany(sql_property, property_records)

In [ ]:
conn.commit()

Hay valores de location_id que son nulos

In [ ]:
# 0) Antes de construir records, comprueba si hay pares sin mapear
unique_locs = set(zip(df_property.barrio, df_property.distrito))
missing = [loc for loc in unique_locs if loc not in loc_map]
if missing:
    print("¡Atención! Estas ubicaciones no están en loc_map:")
    for barrio, distrito in missing:
        print("  -", barrio, "|", distrito)
    # Aquí podrías:
    #  a) Insertarlas en la BD y recargar loc_map
    #  b) O descartarlas de df_property
    # Ejemplo (b):
    df_property = df_property[~df_property.apply(
        lambda r: (r.barrio, r.distrito) in missing, axis=1
    )].copy()


¡Atención! Estas ubicaciones no están en loc_map:
  - None | None


In [ ]:
df[df["barrio"].isna() | df["distrito"].isna()]

,price_current,price_kind,listing_type,barrio,distrito,latitude,longitude,energy_consumption_rating,energy_emissions_rating,adaptado a personas con movilidad reducida,...,portero automático,puerta blindada,sistema de seguridad,superficie construida,superficie útil,terraza,trastero,vidrios dobles,referencia,ref_unique
2155,3400.0,rent_month,rental,NaN,NaN,40.506716,-3.647632,False,False,0.0,...,True,True,False,180.0,175.0,False,False,False,1101969,1101969
2156,4400.0,rent_month,rental,NaN,NaN,40.501321,-3.640140,False,False,0.0,...,True,True,True,229.0,204.0,False,False,False,4475-GF,4475-GF
2157,5500.0,rent_month,rental,NaN,NaN,40.507608,-3.646794,False,False,0.0,...,False,False,True,343.0,310.0,False,False,False,1101971,1101971
2158,3200.0,rent_month,rental,NaN,NaN,40.513502,-3.654116,False,False,0.0,...,False,False,False,140.0,140.0,False,False,False,1101534,1101534
2159,4400.0,rent_month,rental,NaN,NaN,40.500691,-3.635682,False,False,0.0,...,False,False,False,309.0,204.0,False,False,False,08617,08617
2160,3000.0,rent_month,rental,NaN,NaN,40.513189,-3.652046,False,False,0.0,...,True,True,False,170.0,150.0,True,False,False,1101891,1101891
2161,3350.0,rent_month,rental,NaN,NaN,40.507814,-3.643431,False,False,0.0,...,False,True,False,180.0,180.0,True,False,False,1101957,1101957
2162,4350.0,rent_month,rental,NaN,NaN,40.512263,-3.650165,False,False,0.0,...,False,False,False,210.0,210.0,False,False,False,I-02Z5O8-W-02Z5O8,I-02Z5O8-W-02Z5O8
2163,2550.0,rent_month,rental,NaN,NaN,40.512968,-3.651277,False,False,1.0,...,False,False,False,124.0,106.0,False,False,False,08596,08596
2164,3500.0,rent_month,rental,NaN,NaN,40.511998,-3.649115,False,False,0.0,...,True,False,True,180.0,153.0,True,False,False,1101955,1101955


In [ ]:
df[df["referencia"] == '000007'] 

,price_current,price_kind,listing_type,barrio,distrito,latitude,longitude,energy_consumption_rating,energy_emissions_rating,adaptado a personas con movilidad reducida,...,portero automático,puerta blindada,sistema de seguridad,superficie construida,superficie útil,terraza,trastero,vidrios dobles,referencia,ref_unique
11,1500.0,rent_month,rental,Justicia-Chueca,Centro,40.425410,-3.699747,False,False,0.0,...,False,False,False,65.0,60.0,False,False,False,000007,000007
233,1150.0,rent_month,rental,Cortes-Huertas,Centro,40.412512,-3.698424,False,False,0.0,...,False,False,False,55.0,50.0,False,False,False,000007,000007_x2
276,2200.0,rent_month,rental,Goya,Salamanca,40.422368,-3.677569,F,F,0.0,...,False,False,False,66.0,59.0,False,False,False,000007,000007_x3


In [ ]:
len(df["referencia"].unique())

5173

In [ ]:
df.shape

(5186, 35)

hay referencias repetidas, así que añadiremos _xn, siendo n el numero de repeticion para poder diferenciarlas

In [ ]:
df["dup_idx"] = df.groupby("referencia").cumcount()
df["dup_idx"]

0       0
1       0
2       0
3       0
4       0
       ..
5182    0
5183    0
5184    0
5185    0
5186    0
Name: dup_idx, Length: 5186, dtype: int64

In [ ]:
df["dup_idx"].value_counts()

dup_idx
0    5173
1      10
2       3
Name: count, dtype: int64

In [ ]:
df["ref_unique"] = np.where(
    df["dup_idx"] == 0,
    df["referencia"],
    df["referencia"] + "_x" + (df["dup_idx"] + 1).astype(str)
)

df = df.drop(columns=["dup_idx"])

In [ ]:
len(df["referencia"].unique())

5173

### mapeo de las ids de la tabla property

In [ ]:
cursor.execute("SELECT property_id, property_native_id FROM property")

In [ ]:
p = cursor.fetchall()
p

[(227, '000002'),
 (746, '000002_x2'),
 (179, '000006'),
 (257, '000006_x2'),
 (12, '000007'),
 (234, '000007_x2'),
 (277, '000007_x3'),
 (272, '000008'),
 (274, '000008_x2'),
 (293, '000009'),
 (303, '000009_x2'),
 (1280, '000009_x3'),
 (177, '000010'),
 (509, '000010_x2'),
 (910, '000010_x3'),
 (176, '000011'),
 (444, '000013'),
 (68, '000016'),
 (525, '000016_x2'),
 (468, '000019'),
 (569, '000022'),
 (1162, '000023'),
 (178, '000025'),
 (25, '000026'),
 (402, '000028'),
 (2122, '000028_x2'),
 (18, '000032'),
 (24, '000035'),
 (391, '000038'),
 (340, '000043'),
 (305, '000045'),
 (190, '000050'),
 (407, '000050_x2'),
 (278, '000051'),
 (532, '000052'),
 (28, '000057'),
 (616, '000221'),
 (589, '0004'),
 (195, '0005'),
 (586, '000548'),
 (77, '000566'),
 (255, '000579'),
 (193, '000654'),
 (83, '000662'),
 (934, '000665'),
 (1033, '000674_A'),
 (169, '000780'),
 (1055, '000805'),
 (811, '000836'),
 (961, '000907'),
 (936, '000987'),
 (907, '000990'),
 (995, '001058'),
 (962, '001067'

In [ ]:
property_map = {native: pid for pid, native in p}
property_map


{'000002': 227,
 '000002_x2': 746,
 '000006': 179,
 '000006_x2': 257,
 '000007': 12,
 '000007_x2': 234,
 '000007_x3': 277,
 '000008': 272,
 '000008_x2': 274,
 '000009': 293,
 '000009_x2': 303,
 '000009_x3': 1280,
 '000010': 177,
 '000010_x2': 509,
 '000010_x3': 910,
 '000011': 176,
 '000013': 444,
 '000016': 68,
 '000016_x2': 525,
 '000019': 468,
 '000022': 569,
 '000023': 1162,
 '000025': 178,
 '000026': 25,
 '000028': 402,
 '000028_x2': 2122,
 '000032': 18,
 '000035': 24,
 '000038': 391,
 '000043': 340,
 '000045': 305,
 '000050': 190,
 '000050_x2': 407,
 '000051': 278,
 '000052': 532,
 '000057': 28,
 '000221': 616,
 '0004': 589,
 '0005': 195,
 '000548': 586,
 '000566': 77,
 '000579': 255,
 '000654': 193,
 '000662': 83,
 '000665': 934,
 '000674_A': 1033,
 '000780': 169,
 '000805': 1055,
 '000836': 811,
 '000907': 961,
 '000987': 936,
 '000990': 907,
 '001058': 995,
 '001067': 962,
 '00222/11014': 2039,
 '003350C': 957,
 '0044-VC': 741,
 '0078': 774,
 '01006/2151': 658,
 '012134': 558,

### Tabla ``listing``

In [ ]:
df_listing = df[["ref_unique", "listing_type", "price_kind", "price_current"]]
df_listing

,ref_unique,listing_type,price_kind,price_current
0,6013,rental,rent_month,10000.0
1,3893,rental,rent_month,1450.0
2,4411,rental,rent_month,2200.0
3,20815,rental,rent_month,1700.0
4,9635001141/9635,rental,rent_month,2650.0
...,...,...,...,...
5182,ref2997,sale,sale_price,279000.0
5183,ref2998,sale,sale_price,411000.0
5184,ref2999,sale,sale_price,1550000.0
5185,ref3000,sale,sale_price,2490000.0


In [ ]:
df_listing = df_listing.rename(columns={"ref_unique":"property_native_id"})
df_listing

,property_native_id,listing_type,price_kind,price_current
0,6013,rental,rent_month,10000.0
1,3893,rental,rent_month,1450.0
2,4411,rental,rent_month,2200.0
3,20815,rental,rent_month,1700.0
4,9635001141/9635,rental,rent_month,2650.0
...,...,...,...,...
5182,ref2997,sale,sale_price,279000.0
5183,ref2998,sale,sale_price,411000.0
5184,ref2999,sale,sale_price,1550000.0
5185,ref3000,sale,sale_price,2490000.0


In [ ]:
list_records = []
for _, row in df_listing.iterrows():
    pid = property_map.get(row.property_native_id)
    if pid is None:
        continue
    list_records.append((
        pid,
        row.listing_type,
        row.price_kind,
        row.price_current
    ))

list_records

[(1, 'rental', 'rent_month', 10000.0),
 (2, 'rental', 'rent_month', 1450.0),
 (3, 'rental', 'rent_month', 2200.0),
 (4, 'rental', 'rent_month', 1700.0),
 (5, 'rental', 'rent_month', 2650.0),
 (6, 'rental', 'rent_month', 1500.0),
 (7, 'rental', 'rent_month', 1100.0),
 (8, 'rental', 'rent_month', 980.0),
 (9, 'rental', 'rent_month', 1000.0),
 (10, 'rental', 'rent_month', 2000.0),
 (11, 'rental', 'rent_month', 2600.0),
 (12, 'rental', 'rent_month', 1500.0),
 (13, 'rental', 'rent_month', 3950.0),
 (14, 'rental', 'rent_month', 1600.0),
 (15, 'rental', 'rent_month', 3500.0),
 (16, 'rental', 'rent_month', 9000.0),
 (17, 'rental', 'rent_month', 2700.0),
 (18, 'rental', 'rent_month', 1000.0),
 (19, 'rental', 'rent_month', 1100.0),
 (20, 'rental', 'rent_month', 1695.0),
 (21, 'rental', 'rent_month', 1600.0),
 (22, 'rental', 'rent_month', 2495.0),
 (23, 'rental', 'rent_month', 1364.0),
 (24, 'rental', 'rent_month', 1200.0),
 (25, 'rental', 'rent_month', 1200.0),
 (26, 'rental', 'rent_month', 3290

In [ ]:
sql_listing = """
            INSERT INTO listing
            (property_id, listing_type, price_kind, price_current)
            VALUES
            (%s,%s,%s,%s)
      """

In [ ]:
cursor.executemany(sql_listing, list_records)

In [ ]:
conn.commit()

In [ ]:
cursor.rowcount

5155

In [ ]:
df_listing.shape

(5155, 4)

#### Mapeo de la id de la tabla listing

In [ ]:
cursor.execute("""
  SELECT l.listing_id, p.property_native_id
    FROM listing l
    JOIN property p USING(property_id)
""")

In [ ]:
l = cursor.fetchall()
l

[(227, '000002'),
 (746, '000002_x2'),
 (179, '000006'),
 (257, '000006_x2'),
 (12, '000007'),
 (234, '000007_x2'),
 (277, '000007_x3'),
 (272, '000008'),
 (274, '000008_x2'),
 (293, '000009'),
 (303, '000009_x2'),
 (1280, '000009_x3'),
 (177, '000010'),
 (509, '000010_x2'),
 (910, '000010_x3'),
 (176, '000011'),
 (444, '000013'),
 (68, '000016'),
 (525, '000016_x2'),
 (468, '000019'),
 (569, '000022'),
 (1162, '000023'),
 (178, '000025'),
 (25, '000026'),
 (402, '000028'),
 (2122, '000028_x2'),
 (18, '000032'),
 (24, '000035'),
 (391, '000038'),
 (340, '000043'),
 (305, '000045'),
 (190, '000050'),
 (407, '000050_x2'),
 (278, '000051'),
 (532, '000052'),
 (28, '000057'),
 (616, '000221'),
 (589, '0004'),
 (195, '0005'),
 (586, '000548'),
 (77, '000566'),
 (255, '000579'),
 (193, '000654'),
 (83, '000662'),
 (934, '000665'),
 (1033, '000674_A'),
 (169, '000780'),
 (1055, '000805'),
 (811, '000836'),
 (961, '000907'),
 (936, '000987'),
 (907, '000990'),
 (995, '001058'),
 (962, '001067'

In [ ]:
listing_map = { native: lid for lid, native in l }
listing_map

{'000002': 227,
 '000002_x2': 746,
 '000006': 179,
 '000006_x2': 257,
 '000007': 12,
 '000007_x2': 234,
 '000007_x3': 277,
 '000008': 272,
 '000008_x2': 274,
 '000009': 293,
 '000009_x2': 303,
 '000009_x3': 1280,
 '000010': 177,
 '000010_x2': 509,
 '000010_x3': 910,
 '000011': 176,
 '000013': 444,
 '000016': 68,
 '000016_x2': 525,
 '000019': 468,
 '000022': 569,
 '000023': 1162,
 '000025': 178,
 '000026': 25,
 '000028': 402,
 '000028_x2': 2122,
 '000032': 18,
 '000035': 24,
 '000038': 391,
 '000043': 340,
 '000045': 305,
 '000050': 190,
 '000050_x2': 407,
 '000051': 278,
 '000052': 532,
 '000057': 28,
 '000221': 616,
 '0004': 589,
 '0005': 195,
 '000548': 586,
 '000566': 77,
 '000579': 255,
 '000654': 193,
 '000662': 83,
 '000665': 934,
 '000674_A': 1033,
 '000780': 169,
 '000805': 1055,
 '000836': 811,
 '000907': 961,
 '000987': 936,
 '000990': 907,
 '001058': 995,
 '001067': 962,
 '00222/11014': 2039,
 '003350C': 957,
 '0044-VC': 741,
 '0078': 774,
 '01006/2151': 658,
 '012134': 558,

### Tabla `rental_term`

esta tabla la borraremos

### Tabla `energy_certificate`

In [ ]:
df_energy = df[["ref_unique","energy_consumption_rating","energy_emissions_rating"]]
df_energy

,ref_unique,energy_consumption_rating,energy_emissions_rating
0,6013,False,False
1,3893,False,False
2,4411,False,False
3,20815,False,False
4,9635001141/9635,D,D
...,...,...,...
5182,ref2997,E,E
5183,ref2998,False,False
5184,ref2999,D,D
5185,ref3000,False,False


In [ ]:
df_energy["energy_consumption_rating"].unique()

array(['False', 'D', 'E', 'C', 'F', 'G', 'B', 'A'], dtype=object)

In [349]:
df_energy["energy_emissions_rating"].unique()

array(['False', 'D', 'E', 'C', 'G', 'F', 'B', 'A'], dtype=object)

In [351]:
df_energy["energy_consumption_rating"] = df_energy["energy_consumption_rating"].replace({"False": None})
df_energy["energy_consumption_rating"]

C:\Users\marta\AppData\Local\Temp\ipykernel_25340\3944645973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_energy["energy_consumption_rating"] = df_energy["energy_consumption_rating"].replace({"False": None})


0       None
1       None
2       None
3       None
4          D
        ... 
5182       E
5183    None
5184       D
5185    None
5186    None
Name: energy_consumption_rating, Length: 5155, dtype: object

In [352]:
df_energy["energy_emissions_rating"] = df_energy["energy_emissions_rating"].replace({"False": None})
df_energy["energy_emissions_rating"]

C:\Users\marta\AppData\Local\Temp\ipykernel_25340\3486043063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_energy["energy_emissions_rating"] = df_energy["energy_emissions_rating"].replace({"False": None})


0       None
1       None
2       None
3       None
4          D
        ... 
5182       E
5183    None
5184       D
5185    None
5186    None
Name: energy_emissions_rating, Length: 5155, dtype: object

Lista de tuplas a insertar

In [358]:
set(df["ref_unique"].unique()) - set(property_map.keys())

set()

In [360]:
df_energy[(df_energy["energy_consumption_rating"] == None) & (df_energy["energy_emissions_rating"] == None)]

,ref_unique,energy_consumption_rating,energy_emissions_rating


In [363]:
energy_records = []
for _, row in df_energy.iterrows():
    pid = property_map.get(row.ref_unique)
    consumo  = row.energy_consumption_rating
    emisiones = row.energy_emissions_rating

    if consumo is None and emisiones is None:
        continue

    energy_records.append((pid, consumo, emisiones))

energy_records

[(5, 'D', 'D'),
 (15, 'E', 'D'),
 (20, 'E', 'E'),
 (22, 'C', 'C'),
 (26, 'E', 'E'),
 (31, 'C', 'C'),
 (32, 'E', 'E'),
 (43, 'F', 'E'),
 (58, 'G', 'G'),
 (64, 'E', 'E'),
 (72, 'E', 'F'),
 (76, 'D', 'D'),
 (78, 'D', 'D'),
 (79, 'D', 'C'),
 (86, 'E', 'D'),
 (97, 'B', 'B'),
 (99, 'E', 'D'),
 (108, 'B', 'B'),
 (122, 'E', 'E'),
 (128, 'E', 'E'),
 (129, 'D', 'D'),
 (137, 'E', 'E'),
 (141, 'E', 'E'),
 (143, 'D', 'D'),
 (149, 'E', 'E'),
 (153, 'C', 'C'),
 (158, 'B', 'B'),
 (163, 'F', 'F'),
 (166, 'E', 'D'),
 (169, 'C', 'C'),
 (175, 'E', 'G'),
 (184, 'E', 'E'),
 (194, 'E', 'E'),
 (199, 'E', 'D'),
 (215, 'B', 'B'),
 (216, 'B', 'B'),
 (217, 'B', 'B'),
 (219, 'B', 'B'),
 (220, 'B', 'B'),
 (223, 'B', 'B'),
 (226, 'E', 'E'),
 (228, 'E', 'E'),
 (232, 'B', 'B'),
 (236, 'B', 'B'),
 (242, 'E', 'E'),
 (254, 'E', 'E'),
 (256, 'D', 'D'),
 (258, 'B', 'B'),
 (270, 'E', 'E'),
 (277, 'F', 'F'),
 (282, 'D', 'D'),
 (285, 'D', 'D'),
 (291, 'E', 'E'),
 (293, 'E', 'E'),
 (297, 'E', 'E'),
 (298, 'E', 'E'),
 (299, 'E'

In [364]:
sql_energy = """
                INSERT INTO energy_certificate
                (property_id, consumo_rating, emisiones_rating)
                VALUES
                (%s,%s,%s)
            """

In [365]:
cursor.executemany(sql_energy, energy_records)

In [366]:
conn.commit()

### Tabla `property_feature`

In [367]:
bool_cols = [
    "ascensor","balcón","calefacción","chimenea","exterior","garaje",
    "piscina","trastero","jardín","adaptado a personas con movilidad reducida",
    "aire acondicionado","puerta blindada","vidrios dobles",
    "sistema de seguridad","terraza"
]

In [368]:
col_to_feat = {
    "ascensor": "ascensor",
    "balcón": "balcon",
    "calefacción": "calefaccion",
    "chimenea": "chimenea",
    "exterior": "exterior",
    "garaje": "garaje",
    "piscina": "piscina",
    "trastero": "trastero",
    "jardín": "jardin",
    "adaptado a personas con movilidad reducida": "adaptado_pmreducida",
    "aire acondicionado": "aire_acondicionado",
    "puerta blindada": "puerta_blindada",
    "vidrios dobles": "vidrios_dobles",
    "sistema de seguridad": "sistema_seguridad",
    "terraza": "terraza"
}

In [369]:
df["amueblado"].unique()

array(['No', 'Si', 'Semi'], dtype=object)

In [370]:
[df[columna].unique() for columna in bool_cols]

[array([False,  True]),
 array(['False', '1.0'], dtype=object),
 array([ True, False]),
 array(['False', '1.0', 'True', 'Preinstalación'], dtype=object),
 array([False,  True]),
 array([ True, False]),
 array([False,  True]),
 array([False,  True]),
 array([False,  True]),
 array([0., 1.]),
 array([ True, False]),
 array(['False', 'True', 'Acorazada.', 'Puerta acorazada y alarma',
        'Acorazada', 'Entrada'], dtype=object),
 array([False,  True]),
 array([False,  True]),
 array([False,  True])]

In [372]:
df["puerta blindada"].unique()

array(['False', 'True', 'Acorazada.', 'Puerta acorazada y alarma',
       'Acorazada', 'Entrada'], dtype=object)

In [373]:
df["puerta blindada"].value_counts()

puerta blindada
False                        4410
True                          739
Acorazada.                      2
Acorazada                       2
Puerta acorazada y alarma       1
Entrada                         1
Name: count, dtype: int64

In [374]:
m = {
    "False": False,
    "True": True,
    "Acorazada.": True,
    "Acorazada": True,
    "Puerta acorazada y alarma": True,
    "Entrada": False 
}

In [375]:
df["puerta blindada"] = df["puerta blindada"].map(m)
df["puerta blindada"].value_counts()

puerta blindada
False    4411
True      744
Name: count, dtype: int64

In [377]:
df["chimenea"].value_counts()

chimenea
False             5061
True                69
1.0                 24
Preinstalación       1
Name: count, dtype: int64

In [384]:
for d in (df_rental, df_sale):
    d["dup_idx"] = d.groupby("referencia").cumcount()
    d["ref_unique"] = np.where(
        d["dup_idx"] == 0,
        d["referencia"],
        d["referencia"] + "_x" + (d["dup_idx"] + 1).astype(str)
    )
    d.drop(columns=["dup_idx"], inplace=True)

In [385]:
df_chim = pd.concat([
    df_rental[["ref_unique","chimenea"]],
    df_sale  [ ["ref_unique","chimenea"]]
], ignore_index=True)

In [387]:
df_chim["chimenea"].value_counts()

chimenea
False             5086
True                70
1.0                 30
Preinstalación       1
Name: count, dtype: int64

In [391]:
def parse_bool(val):
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return None
    s = str(val).strip().lower()
    if s in ("true","1","1.0","sí","si"):
        return True
    if s in ("false","0","0.0","no"):
        return False
    return None

In [393]:
df_chim["chimenea"] = df_chim["chimenea"].apply(parse_bool)

In [394]:
df_chim = df_chim.drop_duplicates(subset="ref_unique")

In [395]:
df = df.drop(columns=["chimenea"], errors="ignore")  # si df tenía chimenea vacía
df = df.merge(df_chim, on="ref_unique", how="left")

In [396]:
df["chimenea"].value_counts()

chimenea
False    5061
True       93
Name: count, dtype: int64

In [397]:
[df[columna].unique() for columna in bool_cols]

[array([False,  True]),
 array(['False', '1.0'], dtype=object),
 array([ True, False]),
 array([False, True, None], dtype=object),
 array([False,  True]),
 array([ True, False]),
 array([False,  True]),
 array([False,  True]),
 array([False,  True]),
 array([0., 1.]),
 array([ True, False]),
 array([False,  True]),
 array([False,  True]),
 array([False,  True]),
 array([False,  True])]

In [400]:
df["property_id"] = df["ref_unique"].map(property_map)
df["property_id"]

0          1
1          2
2          3
3          4
4          5
        ... 
5150    5151
5151    5152
5152    5153
5153    5154
5154    5155
Name: property_id, Length: 5155, dtype: int64

In [ ]:
pf_records = []
for _, row in df.iterrows():
    pid = property_map.get(row.ref_unique)
    if pid is None:
        continue

    for col, feat_key in col_to_feat.items():
        if row.get(col) is True:
            fid = feature_map.get(feat_key)
            if fid:
                pf_records.append((pid, fid, "true"))

    am = row.get("amueblado")
    if am in ("no", "semi", "si"):
        fid = feature_map.get("amueblado")
        if fid:
            pf_records.append((pid, fid, am))

pf_records

[(1, 6, 'true'),
 (1, 11, 'true'),
 (4, 12, 'true'),
 (6, 7, 'true'),
 (7, 12, 'true'),
 (7, 14, 'true'),
 (8, 12, 'true'),
 (9, 6, 'true'),
 (10, 6, 'true'),
 (10, 11, 'true'),
 (10, 12, 'true'),
 (11, 5, 'true'),
 (11, 11, 'true'),
 (12, 11, 'true'),
 (13, 4, 'true'),
 (13, 6, 'true'),
 (13, 7, 'true'),
 (13, 11, 'true'),
 (13, 12, 'true'),
 (14, 11, 'true'),
 (14, 12, 'true'),
 (14, 14, 'true'),
 (16, 4, 'true'),
 (16, 11, 'true'),
 (17, 11, 'true'),
 (17, 12, 'true'),
 (17, 14, 'true'),
 (18, 11, 'true'),
 (18, 12, 'true'),
 (18, 15, 'true'),
 (20, 6, 'true'),
 (20, 12, 'true'),
 (25, 7, 'true'),
 (25, 12, 'true'),
 (27, 5, 'true'),
 (27, 11, 'true'),
 (27, 12, 'true'),
 (28, 11, 'true'),
 (29, 6, 'true'),
 (29, 7, 'true'),
 (29, 11, 'true'),
 (29, 12, 'true'),
 (31, 6, 'true'),
 (33, 11, 'true'),
 (33, 12, 'true'),
 (34, 12, 'true'),
 (35, 12, 'true'),
 (36, 11, 'true'),
 (39, 11, 'true'),
 (41, 11, 'true'),
 (42, 11, 'true'),
 (42, 12, 'true'),
 (44, 11, 'true'),
 (45, 11, 'true'

In [402]:
sql_pf = """
            INSERT INTO property_feature
                (property_id, feature_id, valor)
            VALUES
                (%s,%s,%s)
        """

In [403]:
cursor.executemany(sql_pf, pf_records)

In [404]:
conn.commit()

In [405]:
cursor.close()
conn.close() 

In [47]:
columnas2 = df_sale.columns.tolist()
columnas2

['price_eur',
 'barrio',
 'distrito',
 'latitude',
 'longitude',
 'energy_consumption_rating',
 'energy_consumption_value',
 'energy_emissions_rating',
 'energy_emissions_value',
 'adaptado a personas con movilidad reducida',
 'aire acondicionado',
 'amueblado',
 'antigüedad',
 'ascensor',
 'balcón',
 'baños',
 'calefacción',
 'chimenea',
 'conservación',
 'exterior',
 'garaje',
 'habitaciones',
 'jardín',
 'piscina',
 'planta',
 'portero automático',
 'puerta blindada',
 'sistema de seguridad',
 'superficie construida',
 'superficie útil',
 'terraza',
 'trastero',
 'vidrios dobles']

In [15]:
# Convertir DataFrame a lista de tuplas
datos = [
    tuple(row[col] if pd.notna(row[col]) else None for col in columnas)
    for _, row in df.iterrows()
]

In [28]:
# Convertir DataFrame a lista de tuplas
datos2 = [
    tuple(row[col] if pd.notna(row[col]) else None for col in columnas2)
    for _, row in df2.iterrows()
]

In [18]:
# Insertar todos los datos de golpe
cursor.executemany(sql, datos)
conn.commit()


In [29]:
# Insertar todos los datos de golpe
cursor2.executemany(sql2, datos2)
conn2.commit()

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'a personas con movilidad reducida, Agua, Aire acondicionado, Amueblado, Antigüe' at line 1

In [19]:
cursor.execute("SHOW TABLES")
for (table_name,) in cursor.fetchall():
    print(table_name)

countries


Insertar la tabla location

In [48]:
df_sale[["barrio","distrito","latitude","latitude"]]

,barrio,distrito,latitude,latitude
0,Recoletos,Salamanca,40.422116,40.422116
1,Universidad-Malasaña,Centro,40.422312,40.422312
2,Goya,Salamanca,40.424364,40.424364
3,Opañel,Carabanchel,40.389100,40.389100
4,Rejas,San Blas,40.442042,40.442042
...,...,...,...,...
2997,Portazgo,Puente de Vallecas,40.383712,40.383712
2998,Pueblo Nuevo,Ciudad Lineal,40.426333,40.426333
2999,Valdemarín,Moncloa-Aravaca,40.465343,40.465343
3000,Trafalgar,Chamberí,40.435597,40.435597


In [49]:
df_location = df_sale[["barrio","distrito","latitude","longitude"]]
df_location

,barrio,distrito,latitude,longitude
0,Recoletos,Salamanca,40.422116,-3.683149
1,Universidad-Malasaña,Centro,40.422312,-3.706137
2,Goya,Salamanca,40.424364,-3.670577
3,Opañel,Carabanchel,40.389100,-3.719900
4,Rejas,San Blas,40.442042,-3.575920
...,...,...,...,...
2997,Portazgo,Puente de Vallecas,40.383712,-3.650452
2998,Pueblo Nuevo,Ciudad Lineal,40.426333,-3.638205
2999,Valdemarín,Moncloa-Aravaca,40.465343,-3.762967
3000,Trafalgar,Chamberí,40.435597,-3.700479


In [50]:
columnas = df_location.columns.to_list()
columnas

['barrio', 'distrito', 'latitude', 'longitude']

In [51]:
# Convertir DataFrame a lista de tuplas
location = [
    tuple(row[col] for col in columnas)
    for _, row in df_location.iterrows()
]

In [52]:
location

[('Recoletos', 'Salamanca', 40.4221164, -3.6831486),
 ('Universidad-Malasaña', 'Centro', 40.422312, -3.706137),
 ('Goya', 'Salamanca', 40.4243638, -3.6705767),
 ('Opañel', 'Carabanchel', 40.3891, -3.7199),
 ('Rejas', 'San Blas', 40.4420421, -3.5759197),
 ('Lista', 'Salamanca', 40.4295857, -3.6735977),
 ('Guindalera', 'Salamanca', 40.4390839, -3.6613366),
 ('Embajadores-Lavapiés', 'Centro', 40.4129747, -3.7059398),
 ('Lucero', 'Latina', 40.4025977, -3.7530822),
 ('Goya', 'Salamanca', 40.4241, -3.67855),
 ('Recoletos', 'Salamanca', 40.4226281, -3.6831171),
 ('El Plantío', 'Moncloa-Aravaca', 40.4767, -3.82927),
 ('Castellana', 'Salamanca', 40.4277347, -3.685881),
 ('Las Águilas', 'Latina', 40.377671, -3.7731945),
 ('Lista', 'Salamanca', 40.4318763, -3.6736524),
 ('Palacio', 'Centro', 40.4118684, -3.7176348),
 ('El Cañaveral', 'Vicálvaro', 40.411542, -3.5447233),
 ('Lista', 'Salamanca', 40.4334893, -3.6773472),
 ('Guindalera', 'Salamanca', 40.434233, -3.6672526),
 ('Guindalera', 'Salamanca

Es importante que los datos estén en el formato adecuado para poder cargar la base de datos, tampoco podemos tener valores nan

In [56]:
sql_location = """
    INSERT INTO location (barrio, distrito, latitude, longitude)
    VALUES (%s, %s, %s, %s)
"""
cursor2.executemany(sql_location, location)
conn2.commit()

Se seguirá la misma estrategia de trabajo expuesta para cargar la base de datos cuando los datos extraidos esten limpios, teniendo en cuenta además que son dos fichero, uno para ventas y otro para alquileres

insertados datos de la tabla location del df de sale

### Tabla property de `sale`

In [73]:
loc_db = pd.read_sql(
    "SELECT location_id, latitude, longitude FROM location", conn2)

C:\Users\marta\AppData\Local\Temp\ipykernel_25340\400070013.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  loc_db = pd.read_sql(


In [77]:
loc_db.short(by="location_id")

AttributeError: 'DataFrame' object has no attribute 'short'

In [69]:
# recuperar ids
query_ids_location = """
SELECT location_id, latitude, longitude
FROM   location
WHERE  (latitude, longitude) IN (%s)
"""

In [71]:
cursor2.executemany(query_ids_location)

TypeError: CMySQLCursor.executemany() missing 1 required positional argument: 'seq_params'

In [60]:
cols_loc = ["barrio","distrito","latitude","longitude"]
df_loc   = df_sale[cols_loc].copy()

In [61]:
start_id  = cursor2.lastrowid - len(df_loc) + 1
start_id

-3000

In [65]:
cursor2.lastrowid

1

In [62]:
import numpy as np

In [63]:
df_loc["location_id"] = np.arange(start_id, start_id + len(df_loc))


In [64]:
df_loc["location_id"]

0      -3000
1      -2999
2      -2998
3      -2997
4      -2996
        ... 
2997      -3
2998      -2
2999      -1
3000       0
3001       1
Name: location_id, Length: 3002, dtype: int32